In [11]:
# -*- coding: utf-8 -*-
import pandas as pd
import random
import csv

from deap import base
from deap import creator
from deap import tools
from deap import cma

import time
start = time.time()


## メンバークラス ##
# インスタンス生成
# 個人属性を設定する
class Member(object):
    def __init__(self, df):
        self.name = df['name']
        self.total = df['total']


## チーム分けクラス ##
class Team():
    def __init__(self, list):
        # 設定
        self.TEAM_N = 3 # チーム数
        self.PEOPLE = 15 # メンバー数
        self.TEAM_P_N = self.PEOPLE / self.TEAM_N # 各チームの人数 # 5

        if list == None:
            self.make_sample()
        else:
            self.list = list
        self.member = []

    # ランダムな遺伝子を生成
    def make_sample(self):
        sample_list = []
        N = self.TEAM_N * self.PEOPLE
        for num in range(N):
            sample_list.append(random.randint(0, 1))
        self.list = tuple(sample_list)


    # 画面表示 #
    # 各チームのポイント合計を表示する
    def get_point(self):
        point = []
        for t in range(self.TEAM_N):
            team = self.get_user(t)
            p = 0
            for m in team:
                p += m.total
            point.append(p)
        print(f'各チームの合計ポイント: {point}')


    # 操作 #
    # タプルを1ユーザ単位に分割
    def slice(self):
        sliced = []
        start = 0
        for num in range(self.PEOPLE):  # 人数 15
            sliced.append(self.list[start:(start + self.TEAM_N)])
            start = start + self.TEAM_N  # チーム数 3
        #print(sliced)
        return tuple(sliced)

    # 指定されたチームにアサインされているユーザリストを返す
    def get_user(self, t):
        m = self.slice()
        count = 0
        team = []
        for i in m:
            if i[t] == 1:
                team.append(self.member[count])
            count += 1
        #print(f'{t}チームの人数は{len(team)}')
        return team


    ###### 以下の目的関数3つを実装してください ######
    
    # 目的関数 #
    # 1人1チームに割当されているか
    def eval1(self):
        return 1

    # 各チームの人数が5人になっているか
    def eval2(self):
        return 1

    # 各チームのスコア獲得ポイント合計は平坦か
    def eval3(self):
        return 1


    

# menberのインスタンス生成
df = pd.read_csv('../input/得点表.csv')
#print(df)

m_list = []
for index, row in df.iterrows():
    tmp = Member(row)
    m_list.append(tmp)

'''
for i in m_list:
    print(i.name)
'''


# 適応度の定義 #
# weights：evalShift関数の条件（目的関数）5つに重み付けしている
creator.create("FitnessPeopleCount", base.Fitness, weights=(-10.0,)) # 変更点
# 個体の定義 #
creator.create("Individual", list, fitness=creator.FitnessPeopleCount)

toolbox = base.Toolbox()

# 遺伝子を作成する関数 #
toolbox.register("attr_bool", random.randint, 0, 1)
# 個体を作成する関数 #
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 45) # 染色体を45個性性
# 世代を作成する関数 #
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



# シフトの評価関数(目的関数)　 ## 変更点
def evalShift(individual):
    t = Team(individual)
    t.member = m_list

    # 目的関数
    eval1 = t.eval1()

    return (eval1,)




## 目的関数、また交差・突然変異・選択を求める関数を読みかえる
# 目的関数 #
toolbox.register("evaluate", evalShift)
# 交叉関数(二点交叉) #
toolbox.register("mate", tools.cxTwoPoint)
# 突然変異関数 #
# indpb=0.05というデフォルト値を与えています。遺伝子突然変異率は5%で決め打ちという事です。
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
# 選択関数(トーナメント選択、tournsizeはトーナメントのサイズ) #
toolbox.register("select", tools.selTournament, tournsize=3)




#random.seed(64)

# 初期集団を生成する
# n=300で世代内の個体数を指定
pop = toolbox.population(n=300)  ## 変更点
# 交叉率、個体突然変異率、ループを回す世代数を指定
CXPB, MUTPB, NGEN = 0.6, 0.4, 1000 ## 変更点

print("進化開始")

# 初期集団の個体を評価する
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):  # zipは複数変数の同時ループ
    # 適合性をセットする
    ind.fitness.values = fit

#print("  %i の個体を評価" % len(pop))

 # 進化計算開始
for g in range(NGEN):
    if g % 100 == 0:
        print("-- %i 世代 --" % g)

    # 選択 #
    # 次世代の個体群を選択
    offspring = toolbox.select(pop, len(pop))
    # 個体群のクローンを生成
    offspring = list(map(toolbox.clone, offspring))

    # 選択した個体群に交差と突然変異を適応する

    # 交叉
    # 偶数番目と奇数番目の個体を取り出して交叉
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # 突然変異
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 適合度が計算されていない個体を集めて適合度を計算
    # 交叉と突然変異でdelした適応度を再計算
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    #print("  %i の個体を評価" % len(invalid_ind))

    pop[:] = offspring

    '''
    # すべての個体の適合度を配列にする
    index = 1
    for v in ind.fitness.values:
        fits = [v for ind in pop]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print(("* パラメータ%d") % index)
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        index += 1
    '''

print("-- 進化終了 --")

best_ind = tools.selBest(pop, 1)[0]
#print("最も優れていた個体: %s, %s" % (best_ind, best_ind.fitness.values))


# チーム編成表示
t = Team(best_ind)
t.member = m_list
#t.print_csv()
#t.print_tsv()
for i in range(t.TEAM_N):
    team = t.get_user(i)
    print(f'チーム{i+1}:')
    count = 1
    for m in team:
        print(f'{count}:{m.name}, {m.total}')
        count += 1
t.get_point()

process_time = time.time() - start
print(f'実行時間:{process_time/60}')

進化開始
-- 0 世代 --
-- 100 世代 --
-- 200 世代 --
-- 300 世代 --
-- 400 世代 --
-- 500 世代 --
-- 600 世代 --
-- 700 世代 --
-- 800 世代 --
-- 900 世代 --
-- 進化終了 --
チーム1:
1:Sato, 3
2:Muto, 3
3:Shima, 5
4:Otsuka, 1
5:Hamada, 5
6:Nakamura, 4
7:Koike, 2
8:Matsui, 4
9:Watanabe, 4
チーム2:
1:Kimura, 0
2:Shima, 5
3:Otsuka, 1
4:Hamada, 5
5:Nakamura, 4
6:Fujino, 3
7:Fujita, 5
チーム3:
1:Nadaoka, 5
2:Kimura, 0
3:Shima, 5
4:Otsuka, 1
5:Hamada, 5
6:Nakamura, 4
7:Matsui, 4
8:Fujita, 5
各チームの合計ポイント: [31, 23, 29]
実行時間:0.19730316797892253
